# 2. Základy knihovny physt

In [ ]:
# Nutné importy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Pojďme si načíst data z předchozího příkladu (ale "jenom" milion)
teploty = pd.read_csv("data/vsechny_teploty.csv.gz", squeeze=True, dtype=np.float32, header=None, nrows=1100000)

# Opět omezíme teploty na "rozumné" rozmezí
teploty = teploty[teploty > -50]
teploty = teploty[teploty < 60]
teploty = teploty[:1000000]

teploty.name = "Teplota"

In [ ]:
teploty.head()

In [ ]:
import physt
from physt import h1

## Objektová reprezentace

Nejjednodušší histogram

In [ ]:
H = physt.h1(teploty)
H

In [ ]:
# Odpovídá stejnému volání v numpy
H.numpy_like()

In [ ]:
# Trošku vivisekce histogramu neuškodí
H.to_dict()

In [ ]:
H.plot()

## "inteligentní" biny

### Pevná šířka

In [ ]:
H = physt.h1(teploty, "fixed_width", bin_width=1, adaptive=True )
H.plot()
H.numpy_like()

In [ ]:
H._binning.to_dict()

In [ ]:
# Na úrovni nejnižšího rozlišení
H = physt.h1(teploty, "fixed_width", bin_width=.1, adaptive=True, range=(-5, 5))
H.plot()

To nebude dobře, začínáme si hrát se zaokrouhlovací chybou!

Trochu to ošidíme (pro počítač je šesté desetinné místo zajímavé), pro nás ne.

In [ ]:
H = physt.h1(teploty, "fixed_width", bin_width=.099999, adaptive=True, range=(-5, 5))
H.plot()

**Varování**: Vyhýbejte se hodnotám na hranici binu a mějte opravdu "float" data

In [ ]:
H = physt.h1(teploty, "fixed_width", bin_width=.099999, bin_shift=0.001, adaptive=True)
H.plot("line")

### Automatická vhodná šířka ("human")

In [ ]:
H = physt.h1(teploty, "human")
H.plot(show_values=True, value_format=lambda x:int(x / 1000))
H.numpy_bins

In [ ]:
for n in [3, 7, 10, 15, 20, 40, 50]:
    H = physt.h1(teploty, "human", n)
    print(n, H.bin_count, H._binning.bin_width)

### Biny o stejném obsahu

In [ ]:
H = physt.h1(teploty, "quantile", 30)    
H.plot()     # Mělo by být stejně široké! Ale není
H.numpy_like()

In [ ]:
physt.h1(teploty, "quantile", 30).plot(density=True)

### Exponenciální

In [ ]:
# Teploty (zatím) nejsou exponenciální, tak jiná data:
countries = pd.read_csv("https://www.worlddata.info/download/countries.csv", delimiter=";")
countries = countries[(countries.Population > 0) & (countries.Area > 0)]

In [ ]:
h1(countries.Population).plot()

In [ ]:
h1(countries.Population, "exponential").plot()

## Plnění histogramů

In [ ]:
np.random.seed(42)
H = physt.h1(None, "fixed_width", 5, adaptive=True)
H

In [ ]:
H.fill_n(np.random.choice(teploty, 1000))
H.plot("bar", errors=True, density=True)
H